
**Environment setup**


In [2]:
%pip install "torch>=2" torchvision xformers
%pip install open3d opencv-python pillow numpy



INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 94.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [5]:
!pip -q uninstall -y depth-anything-3
!pip -q install --upgrade pip


In [6]:
# Colab-safe NumPy range that keeps TF + numba happy
!pip -q install --force-reinstall "numpy==2.0.2"

# Fix gradio pillow constraint
!pip -q install --force-reinstall "pillow<12"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.1 which is incompatible.


Install OpenCV + Open3D + Depth Anything 3

In [8]:
!pip -q install --upgrade opencv-python-headless
!pip -q install --upgrade open3d

# install DA3 but do NOT let it change numpy
!pip -q install git+https://github.com/ByteDance-Seed/Depth-Anything-3.git --no-deps


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Verify imports

In [6]:
import numpy as np
import cv2
import open3d as o3d
import torch

from depth_anything_3.api import DepthAnything3

print("numpy:", np.__version__)
print("opencv:", cv2.__version__)
print("open3d:", o3d.__version__)
print("torch:", torch.__version__)
print("DA3 import OK")


numpy: 2.0.2
opencv: 4.11.0
open3d: 0.19.0
torch: 2.9.0+cu128
DA3 import OK


Import image

In [7]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
print("Image:", image_path)


Saving 20260112_171320.jpg to 20260112_171320 (1).jpg
Image: 20260112_171320 (1).jpg


DA3 interface

In [8]:
import numpy as np
import torch
from depth_anything_3.api import DepthAnything3

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

model_id = "depth-anything/DA3NESTED-GIANT-LARGE"
model = DepthAnything3.from_pretrained(model_id).to(device)

pred = model.inference([image_path])
depth = pred.depth[0].astype(np.float32)

print("depth:", depth.shape, "min/max:", float(depth.min()), float(depth.max()))


device: cuda
[INFO ] using SwiGLU layer as FFN
[INFO ] using MLP layer as FFN
[INFO ] Processed Images Done taking 1.1279354095458984 seconds. Shape:  torch.Size([1, 3, 504, 378])
[INFO ] Model Forward Pass Done. Time: 3.21169376373291 seconds
[INFO ] Conversion to Prediction Done. Time: 0.001295328140258789 seconds
depth: (504, 378) min/max: 1.1107319593429565 1.2722529172897339


Normalize + save depth preview

In [4]:
import cv2
import numpy as np

d = depth - np.nanmin(depth)
p99 = np.nanpercentile(d, 99.0)
d = np.clip(d / (p99 + 1e-6), 0.0, 1.0)

max_depth_m = 2.0
depth_m = d * max_depth_m

cv2.imwrite("depth_map.png", (d * 255).astype(np.uint8))
print("Saved depth_map.png")


Saved depth_map.png


Depth → point cloud → mesh (Open3D)

In [8]:
import open3d as o3d
import cv2
import numpy as np

# ---- Load original image ----
bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
H, W = rgb.shape[:2]

# ---- Downscale for sanity (VERY IMPORTANT for 8K images) ----
target_width = 1200   # 800–1500 is ideal for Colab
scale = target_width / W
new_w = target_width
new_h = int(H * scale)

rgb_small = cv2.resize(rgb, (new_w, new_h), interpolation=cv2.INTER_AREA)
depth_small = cv2.resize(depth_m, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

print("Downscaled size:", rgb_small.shape)

# ---- Convert depth to mm ----
depth_mm = np.clip(depth_small * 1000.0, 0, 65535).astype(np.uint16)

color_o3d = o3d.geometry.Image(rgb_small.astype(np.uint8))
depth_o3d = o3d.geometry.Image(depth_mm)

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d,
    depth_o3d,
    depth_scale=1000.0,
    depth_trunc=10.0,
    convert_rgb_to_intensity=False
)

# ---- Intrinsics must match downscaled resolution ----
h2, w2 = rgb_small.shape[:2]
fx = fy = 0.9 * w2
cx, cy = w2 / 2.0, h2 / 2.0

intr = o3d.camera.PinholeCameraIntrinsic(w2, h2, fx, fy, cx, cy)

# ---- Create point cloud ----
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intr)
print("Initial points:", np.asarray(pcd.points).shape[0])

# ---- Metric downsampling (1 cm voxels) ----
pcd = pcd.voxel_down_sample(voxel_size=0.01)
print("After voxel:", np.asarray(pcd.points).shape[0])

# ---- Light outlier cleanup ----
pcd, _ = pcd.remove_radius_outlier(nb_points=12, radius=0.03)
print("After cleanup:", np.asarray(pcd.points).shape[0])

# ---- Normals ----
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30)
)

# Orient normals toward camera (more stable for single view)
pcd.orient_normals_towards_camera_location(camera_location=np.array([0., 0., 0.]))

print("Point cloud ready:", pcd)

# ---- Poisson reconstruction ----
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9
)

densities = np.asarray(densities)
density_threshold = np.quantile(densities, 0.02)
mesh.remove_vertices_by_mask(densities < density_threshold)

mesh.compute_vertex_normals()

o3d.io.write_triangle_mesh("mesh_output.ply", mesh)
o3d.io.write_triangle_mesh("mesh_output.stl", mesh)

print("Saved mesh_output.ply and mesh_output.stl")


Downscaled size: (1600, 1200, 3)
Initial points: 1919955
After voxel: 50988
After cleanup: 50973
Point cloud ready: PointCloud with 50973 points.
Saved mesh_output.ply and mesh_output.stl


Download

In [9]:
from google.colab import files
files.download("depth_map.png")
files.download("mesh_output.stl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

mask + bilateral + Ball Pivoting

In [10]:
import numpy as np, cv2, open3d as o3d

# --- load rgb and resize both to manageable size ---
bgr = cv2.imread(image_path)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
H, W = rgb.shape[:2]

target_w = 1400
s = target_w / W
w2, h2 = target_w, int(H * s)

rgb2 = cv2.resize(rgb, (w2, h2), interpolation=cv2.INTER_AREA)
d2  = cv2.resize(depth_m, (w2, h2), interpolation=cv2.INTER_LINEAR)

# --- depth-based mask (keep nearer stuff) ---
# adjust these two percentiles depending on your scene
near = np.percentile(d2, 5)
far  = np.percentile(d2, 80)
mask = (d2 < far).astype(np.uint8) * 255

# clean mask
k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)

# apply mask to depth (set background to 0)
d2_masked = d2.copy()
d2_masked[mask == 0] = 0.0

# --- edge-preserving smoothing on depth ---
# bilateral works well for depth maps
d2_smooth = cv2.bilateralFilter(d2_masked.astype(np.float32), d=9, sigmaColor=0.05, sigmaSpace=9)

# convert to mm uint16
depth_mm = np.clip(d2_smooth * 1000.0, 0, 65535).astype(np.uint16)

# --- Open3D RGBD -> point cloud ---
color_o3d = o3d.geometry.Image(rgb2.astype(np.uint8))
depth_o3d = o3d.geometry.Image(depth_mm)

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=1000.0, depth_trunc=10.0, convert_rgb_to_intensity=False
)

fx = fy = 0.9 * w2
cx, cy = w2/2.0, h2/2.0
intr = o3d.camera.PinholeCameraIntrinsic(w2, h2, fx, fy, cx, cy)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intr)

# remove zeros (background)
pts = np.asarray(pcd.points)
valid = np.isfinite(pts).all(axis=1) & (pts[:,2] > 1e-6)
pcd = pcd.select_by_index(np.where(valid)[0])

# downsample in meters
pcd = pcd.voxel_down_sample(0.005)  # 5 mm voxels
pcd, _ = pcd.remove_radius_outlier(nb_points=10, radius=0.02)

pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
pcd.orient_normals_towards_camera_location(np.array([0.0, 0.0, 0.0]))

print("pcd points:", np.asarray(pcd.points).shape[0])

# --- Ball Pivoting (often cleaner than Poisson for single view) ---
radii = [0.005, 0.01, 0.02]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)
mesh.compute_vertex_normals()

o3d.io.write_triangle_mesh("mesh_bpa.ply", mesh)
o3d.io.write_triangle_mesh("mesh_bpa.stl", mesh)
print("Saved mesh_bpa.stl")


pcd points: 94141
Saved mesh_bpa.stl


Depth cleanup + mask (replace your depth->pcd prep)

In [11]:
import numpy as np, cv2

# Load RGB
bgr = cv2.imread(image_path)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
H, W = rgb.shape[:2]

# Work at manageable size
target_w = 1400
s = target_w / W
w2, h2 = target_w, int(H * s)

rgb2 = cv2.resize(rgb, (w2, h2), interpolation=cv2.INTER_AREA)
d2 = cv2.resize(depth_m, (w2, h2), interpolation=cv2.INTER_LINEAR).astype(np.float32)

# Robust normalize depth using percentiles (reduces background dominance)
p1, p99 = np.percentile(d2, 1), np.percentile(d2, 99)
d2 = np.clip((d2 - p1) / (p99 - p1 + 1e-6), 0.0, 1.0)

# Map to a stable "meters-like" span
d2 = 0.25 + d2 * 1.75  # 0.25m..2.0m

# Depth-based mask: keep "closer" region (tune far_pct)
far_pct = 85
far = np.percentile(d2, far_pct)
mask = (d2 < far).astype(np.uint8) * 255

# Clean mask
k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=2)

# Keep largest connected component (removes specks)
num, labels = cv2.connectedComponents(mask)
if num > 1:
    areas = [(labels==i).sum() for i in range(1, num)]
    keep = 1 + int(np.argmax(areas))
    mask = ((labels==keep).astype(np.uint8) * 255)

# Erode mask a bit to avoid halo boundary artifacts
mask_erode = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)), iterations=1)

# Apply masked bilateral smoothing (edge-preserving)
d2_masked = d2.copy()
d2_masked[mask_erode == 0] = 0.0
d2_smooth = cv2.bilateralFilter(d2_masked, d=9, sigmaColor=0.07, sigmaSpace=9)

# Save debug previews
cv2.imwrite("mask.png", mask)
cv2.imwrite("mask_erode.png", mask_erode)
cv2.imwrite("depth_clean.png", (np.clip(d2_smooth / (d2_smooth.max()+1e-6), 0, 1)*255).astype(np.uint8))

print("Saved mask.png, mask_erode.png, depth_clean.png")


Saved mask.png, mask_erode.png, depth_clean.png


Build PCD + BPA mesh + postprocess cleanup

In [12]:
import open3d as o3d
import numpy as np

# Convert depth to uint16 mm
depth_mm = np.clip(d2_smooth * 1000.0, 0, 65535).astype(np.uint16)

color_o3d = o3d.geometry.Image(rgb2.astype(np.uint8))
depth_o3d = o3d.geometry.Image(depth_mm)

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d, depth_scale=1000.0, depth_trunc=10.0, convert_rgb_to_intensity=False
)

h2, w2 = rgb2.shape[:2]
fx = fy = 0.9 * w2
cx, cy = w2/2.0, h2/2.0
intr = o3d.camera.PinholeCameraIntrinsic(w2, h2, fx, fy, cx, cy)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intr)

# Remove background zeros
pts = np.asarray(pcd.points)
valid = np.isfinite(pts).all(axis=1) & (pts[:,2] > 1e-6)
pcd = pcd.select_by_index(np.where(valid)[0])

# Downsample + light denoise
pcd = pcd.voxel_down_sample(0.004)  # 4mm
pcd, _ = pcd.remove_radius_outlier(nb_points=10, radius=0.02)

# Normals
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
pcd.orient_normals_towards_camera_location(np.array([0.0, 0.0, 0.0]))

print("PCD points:", np.asarray(pcd.points).shape[0])

# Ball Pivoting (cleaner than Poisson for single view)
radii = [0.004, 0.008, 0.016]
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

# --- Postprocess mesh ---
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.remove_non_manifold_edges()

# Remove small floating components (keep biggest)
tri_clusters, cluster_n_tri, _ = mesh.cluster_connected_triangles()
cluster_n_tri = np.asarray(cluster_n_tri)
keep_cluster = int(cluster_n_tri.argmax())
tri_clusters = np.asarray(tri_clusters)
mask_tri = tri_clusters != keep_cluster
mesh.remove_triangles_by_mask(mask_tri)
mesh.remove_unreferenced_vertices()

# Gentle smoothing (few iters)
mesh = mesh.filter_smooth_taubin(number_of_iterations=5)
mesh.compute_vertex_normals()

o3d.io.write_triangle_mesh("mesh_clean.stl", mesh)
o3d.io.write_triangle_mesh("mesh_clean.ply", mesh)
print("Saved mesh_clean.stl / mesh_clean.ply")


PCD points: 189293
[Open3D WARNING] Write Ply clamped color value to valid range
Saved mesh_clean.stl / mesh_clean.ply


Relief generator (sliders + rebuild mesh2)

In [9]:
!pip -q install ipywidgets plotly


Enable widgets + set Plotly rendere

In [10]:
from google.colab import output
output.enable_custom_widget_manager()

import plotly.io as pio
pio.renderers.default = "colab"

print("Widgets + Plotly renderer enabled.")


Widgets + Plotly renderer enabled.


INTEGRATED SLIDER + LIVE VIEWER

In [11]:
import numpy as np
import cv2
import plotly.graph_objects as go
from ipywidgets import FloatSlider, Checkbox, IntSlider, VBox, HBox, Output
from IPython.display import display, clear_output

# --- UI ---
relief_s = FloatSlider(min=1, max=40, step=1, value=10, description="relief_mm")
base_s   = FloatSlider(min=0, max=10, step=0.5, value=3, description="base_mm")
px_s     = FloatSlider(min=0.2, max=1.5, step=0.05, value=0.6, description="px_mm")
invert_s = Checkbox(value=False, description="invert")
erode_s  = IntSlider(min=0, max=10, step=1, value=0, description="erode_px")

out = Output()

def build_heightfield_mesh(d2_smooth, mask_erode, relief_mm, base_mm, px_mm, invert, erode_px):
    d = d2_smooth.astype(np.float32).copy()
    mask = (mask_erode > 0).astype(np.uint8)

    if erode_px > 0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*erode_px+1, 2*erode_px+1))
        mask = cv2.erode(mask, k, iterations=1)

    m = mask.astype(bool)
    if m.sum() < 1000:
        raise RuntimeError("Mask too small after erosion. Reduce erode_px or fix mask.")

    vals = d[m]
    p1, p99 = np.percentile(vals, 1), np.percentile(vals, 99)
    dn = np.clip((d - p1) / (p99 - p1 + 1e-6), 0.0, 1.0)
    if invert:
        dn = 1.0 - dn

    H_mm = (base_mm + relief_mm * dn).astype(np.float32)
    H_mm[~m] = 0.0

    h, w = H_mm.shape

    # vertices (meters)
    X = (np.arange(w, dtype=np.float32) - (w-1)/2.0) * (px_mm / 1000.0)
    Y = (np.arange(h, dtype=np.float32) - (h-1)/2.0) * (px_mm / 1000.0)
    XX, YY = np.meshgrid(X, -Y)
    ZZ = H_mm / 1000.0
    verts = np.stack([XX, YY, ZZ], axis=-1).reshape(-1, 3)

    # triangles
    idx = np.arange(h*w).reshape(h, w)
    a = idx[:-1, :-1].reshape(-1)
    b = idx[:-1,  1:].reshape(-1)
    c = idx[ 1:, :-1].reshape(-1)
    d_ = idx[ 1:,  1:].reshape(-1)
    tris1 = np.stack([a, c, b], axis=1)
    tris2 = np.stack([b, c, d_], axis=1)

    good = (m[:-1, :-1] & m[:-1, 1:] & m[1:, :-1] & m[1:, 1:]).reshape(-1)
    faces = np.vstack([tris1[good], tris2[good]]).astype(np.int32)

    return verts, faces

def render(*_):
    with out:
        clear_output(wait=True)
        try:
            verts, faces = build_heightfield_mesh(
                d2_smooth=d2_smooth,
                mask_erode=mask_erode,
                relief_mm=relief_s.value,
                base_mm=base_s.value,
                px_mm=px_s.value,
                invert=invert_s.value,
                erode_px=erode_s.value
            )

            fig = go.Figure(data=[go.Mesh3d(
                x=verts[:,0], y=verts[:,1], z=verts[:,2],
                i=faces[:,0], j=faces[:,1], k=faces[:,2],
                flatshading=True, opacity=1.0
            )])

            fig.update_layout(
                scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode="data"
                ),
                margin=dict(l=0, r=0, b=0, t=0),
                height=650
            )
            display(fig)
        except Exception as e:
            print("Viewer error:", e)

# hook slider changes
for w in [relief_s, base_s, px_s, invert_s, erode_s]:
    w.observe(render, names="value")

ui = VBox([
    HBox([relief_s, base_s]),
    HBox([px_s, invert_s, erode_s]),
    out
])

display(ui)
render()


Reload image + run depth again

In [15]:
import torch
import cv2
import numpy as np
from depth_anything_3.api import DepthAnything3

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# Upload image again if needed
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Run depth model
model_id = "depth-anything/DA3BASE"  # lighter + faster
model = DepthAnything3.from_pretrained(model_id).to(device)

pred = model.inference([image_path])
depth = pred.depth[0].astype(np.float32)

print("Depth ready:", depth.shape)


device: cuda


Saving 20260112_171320.jpg to 20260112_171320 (2).jpg
[INFO ] using MLP layer as FFN


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-698de002-225060540ab4df391995f58b;ca7b6938-44b3-4e71-a7f3-65e2bfc2e126)

Repository Not Found for url: https://huggingface.co/depth-anything/DA3BASE/resolve/main/model.safetensors.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.

Preprocess depth → create d2_smooth + mask_erode

In [16]:
import cv2
import numpy as np

# Resize to manageable resolution
bgr = cv2.imread(image_path)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
H, W = rgb.shape[:2]

target_w = 1400
scale = target_w / W
w2 = target_w
h2 = int(H * scale)

rgb2 = cv2.resize(rgb, (w2, h2), interpolation=cv2.INTER_AREA)
d2 = cv2.resize(depth, (w2, h2), interpolation=cv2.INTER_LINEAR)

# Normalize depth robustly
p1, p99 = np.percentile(d2, 1), np.percentile(d2, 99)
d2 = np.clip((d2 - p1) / (p99 - p1 + 1e-6), 0, 1)

# Depth-based mask
far = np.percentile(d2, 85)
mask = (d2 < far).astype(np.uint8) * 255

# Clean mask
k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k)

mask_erode = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)))

# Smooth depth
d2_smooth = cv2.bilateralFilter(d2.astype(np.float32), 9, 0.05, 9)

print("Preprocessing complete")


Preprocessing complete




Shaded heightfield viewer (with exaggeration)




In [17]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "colab"

relief_mm = 10
px_mm = 0.6
z_exaggeration = 8

m = mask_erode > 0
d = d2_smooth.copy()

vals = d[m]
p1, p99 = np.percentile(vals, 1), np.percentile(vals, 99)
dn = np.clip((d - p1) / (p99 - p1 + 1e-6), 0, 1)

H_mm = relief_mm * dn
H_mm[~m] = 0

h, w = H_mm.shape
X = (np.arange(w) - (w-1)/2) * (px_mm / 1000)
Y = (np.arange(h) - (h-1)/2) * (px_mm / 1000)
XX, YY = np.meshgrid(X, -Y)
ZZ = (H_mm / 1000) * z_exaggeration

verts = np.stack([XX, YY, ZZ], axis=-1).reshape(-1,3)

idx = np.arange(h*w).reshape(h,w)
a = idx[:-1,:-1].reshape(-1)
b = idx[:-1,1:].reshape(-1)
c = idx[1:,:-1].reshape(-1)
d_ = idx[1:,1:].reshape(-1)

tris1 = np.stack([a,c,b],axis=1)
tris2 = np.stack([b,c,d_],axis=1)

good = (m[:-1,:-1] & m[:-1,1:] & m[1:,:-1] & m[1:,1:]).reshape(-1)
faces = np.vstack([tris1[good], tris2[good]])

fig = go.Figure(data=[go.Mesh3d(
    x=verts[:,0], y=verts[:,1], z=verts[:,2],
    i=faces[:,0], j=faces[:,1], k=faces[:,2],
    flatshading=False,
    lighting=dict(ambient=0.2, diffuse=0.8, specular=0.3),
    lightposition=dict(x=2,y=-1,z=3)
)])

fig.update_layout(
    scene=dict(aspectmode='data'),
    margin=dict(l=0,r=0,b=0,t=0),
    height=700
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [23]:
import open3d as o3d
import numpy as np
from google.colab import files

# ---- Parameters ----
relief_mm = 10
base_mm   = 3
px_mm     = 2
invert    = True

# Match the viewer exaggeration (set to what you used, e.g., 6–12)
z_exaggeration_export = 24.0

# ---- Build heightfield ----
m = mask_erode > 0
d = d2_smooth.copy()

vals = d[m]
p1, p99 = np.percentile(vals, 1), np.percentile(vals, 99)
dn = np.clip((d - p1) / (p99 - p1 + 1e-6), 0, 1)

if invert:
    dn = 1 - dn

# IMPORTANT: exaggerate only the relief part
H_mm = base_mm + (relief_mm * z_exaggeration_export) * dn
H_mm[~m] = 0

h, w = H_mm.shape

X = (np.arange(w) - (w-1)/2) * (px_mm / 1000)
Y = (np.arange(h) - (h-1)/2) * (px_mm / 1000)
XX, YY = np.meshgrid(X, -Y)
ZZ = H_mm / 1000

verts = np.stack([XX, YY, ZZ], axis=-1).reshape(-1, 3)

idx = np.arange(h*w).reshape(h, w)
a = idx[:-1, :-1].reshape(-1)
b = idx[:-1,  1:].reshape(-1)
c = idx[ 1:, :-1].reshape(-1)
d_ = idx[ 1:,  1:].reshape(-1)

tris1 = np.stack([a, c, b], axis=1)
tris2 = np.stack([b, c, d_], axis=1)

good = (m[:-1, :-1] & m[:-1, 1:] & m[1:, :-1] & m[1:, 1:]).reshape(-1)
faces = np.vstack([tris1[good], tris2[good]])

mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(verts),
    o3d.utility.Vector3iVector(faces.astype(np.int32))
)
mesh.compute_vertex_normals()

# Save STL
filename = "relief_exaggerated.stl"
o3d.io.write_triangle_mesh(filename, mesh)

print("Saved:", filename)
print("Effective relief height (mm):", relief_mm * z_exaggeration_export)
files.download(filename)


Saved: relief_exaggerated.stl
Effective relief height (mm): 240.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
import open3d as o3d
import numpy as np
import cv2
from google.colab import files

# ---- Parameters ----
relief_mm = 10
base_mm   = 3
px_mm     = 2
invert    = True

z_exaggeration_export = 24.0

# NEW: fix flattening
extra_erode_px = 6     # erode mask more to ignore noisy edges (try 4..12)
use_center_roi = True  # robust stats from central area
detail_boost   = 0.25  # 0..0.6 (adds crispness without warping)

# ---- Mask prep ----
mask = (mask_erode > 0).astype(np.uint8)
if extra_erode_px > 0:
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*extra_erode_px+1, 2*extra_erode_px+1))
    mask = cv2.erode(mask, k, iterations=1)
m = mask.astype(bool)

d = d2_smooth.astype(np.float32).copy()

# ---- Choose normalization region ----
norm_region = m
if use_center_roi:
    h, w = d.shape
    y0, y1 = int(0.15*h), int(0.85*h)
    x0, x1 = int(0.15*w), int(0.85*w)
    center = np.zeros_like(m, dtype=bool)
    center[y0:y1, x0:x1] = True
    norm_region = m & center
    if norm_region.sum() < 1000:
        norm_region = m  # fallback if too small

vals = d[norm_region]
p1, p99 = np.percentile(vals, 1), np.percentile(vals, 99)

dn = np.clip((d - p1) / (p99 - p1 + 1e-6), 0.0, 1.0)
if invert:
    dn = 1.0 - dn

# ---- Optional detail boost (high-pass inside mask) ----
if detail_boost > 0:
    blur = cv2.GaussianBlur(dn, (0,0), sigmaX=3.0)
    hp = dn - blur
    dn = np.clip(dn + detail_boost * hp, 0.0, 1.0)

# ---- Build heightfield ----
H_mm = base_mm + (relief_mm * z_exaggeration_export) * dn
H_mm[~m] = 0.0

h, w = H_mm.shape
X = (np.arange(w) - (w-1)/2) * (px_mm / 1000)
Y = (np.arange(h) - (h-1)/2) * (px_mm / 1000)
XX, YY = np.meshgrid(X, -Y)
ZZ = H_mm / 1000

verts = np.stack([XX, YY, ZZ], axis=-1).reshape(-1, 3)

idx = np.arange(h*w).reshape(h, w)
a = idx[:-1, :-1].reshape(-1)
b = idx[:-1,  1:].reshape(-1)
c = idx[ 1:, :-1].reshape(-1)
d_ = idx[ 1:,  1:].reshape(-1)

tris1 = np.stack([a, c, b], axis=1)
tris2 = np.stack([b, c, d_], axis=1)

good = (m[:-1, :-1] & m[:-1, 1:] & m[1:, :-1] & m[1:, 1:]).reshape(-1)
faces = np.vstack([tris1[good], tris2[good]])

mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(verts),
    o3d.utility.Vector3iVector(faces.astype(np.int32))
)
mesh.compute_vertex_normals()

V = np.asarray(mesh.vertices)
print("Z range (mm):", (V[:,2].max() - V[:,2].min()) * 1000)

filename = "relief_exag24_fixed_mid.stl"
o3d.io.write_triangle_mesh(filename, mesh)
print("Saved:", filename)
files.download(filename)


Z range (mm): 243.00000071525574
Saved: relief_exag24_fixed_mid.stl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>